# L1: NLP 태스크를 간단한 인터페이스로 구현하기 🗞️

HuggingFace API 키, 그리고 관련된 파이썬 라이브러리를 불러옵니다.

In [ ]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [ ]:
# Helper 함수
import requests, json

# 요약 결과
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

## 텍스트 요약 앱 만들기

`facebook/bart-large-cnn` 모델로부터 distilled 되어 306M개 파라미터를 가지는 [추론 엔드포인트](https://huggingface.co/inference-endpoints)를 불러옵니다.  
이 모델의 이름은 `shleifer/distilbart-cnn-12-6`입니다.

### 어떻게 로컬에서 실행할까?
만약 당신이 API 대신 로컬에서 실행한다면, 이 코드는 굉장히 비슷할 것입니다.  
강의 나머지 부분에 등장하는 모든 모델들에 대해 동일한 내용이니, [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) 페이지 내용을 확인해보세요.

```py
from transformers import pipeline

get_completion = pipeline("summarization", model="shleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
```

In [ ]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

### Gradio의 `gr.Interface`로 시작하기

#### 어떻게 로컬에서 실행할까?
먄약 당신에 API 대신 로컬에서 실행한다면 이 코드는 굉장히 비슷할 것입니다.  
launch method에 속하는 모든 파라미터들 제거해보세요.

```py
demo.launch()
```

In [ ]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text") # 추론 인터페이스 구성
demo.launch(share=True, server_port=int(os.environ['PORT1'])) # 추론 인터페이스를 실행

당신의 팀이나 친구들에게 공유할 퍼블릭 링크를 생성하기 위해 `demo.launch(share=True)` 를 추가할 수 있습니다.

In [ ]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True, server_port=int(os.environ['PORT2']))

## 개체명 인식(NER) 앱 만들기

NER 태스크를 위해 BART 모델을 fine-tuning한 108M 개의 파라미터 모델, `dslim/bert-base-NER`의  
[Inference Endpoint](https://huggingface.co/inference-endpoints) 링크입니다.

### 어떻게 로컬에서 실행할까?

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [ ]:
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Andrew, I'm building DeepLearningAI and I live in California"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL) # 여러 딕셔너리를 원소로 갖는 리스트 반환

In [ ]:
def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")], # Entity가 예쁘게 강조됨
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    # 클릭하면 바로 입력으로 쓸 수 있는 예시도 추가할 수 있습니다.
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch(share=True, server_port=int(os.environ['PORT3']))

### 토큰을 합치기 위해 helper 함수 추가하기

위 코드를 실행해보면 시작(Beginning)과 중간(Intermediate) 토큰이 이어집니다.  
이는 항상 완벽하게 구분되는 것은 아니고 모델의 학습 결과에 의한 것이므로 조작이 필요할 때가 있습니다.  
다음 내용은 쪼개진 여러 개의 토큰을 합쳐주는 함수입니다.

In [ ]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # 현재 토큰이 entity의 마지막 토큰이면 합치기
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # 그렇지 않으면 토큰을 리스트에 추가
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch(share=True, server_port=int(os.environ['PORT4']))

In [ ]:
gr.close_all()